# KVASIR Dataset (Simple Neural Network)

Based on https://www.pyimagesearch.com/2018/09/10/keras-tutorial-how-to-get-started-with-keras-deep-learning-and-python/

## Loading dataset from disk

Define data and labels. Here data[i] will contain resized and preprocessed images and label[i] will contain corresponding label for data[i].

In [1]:
data = []
labels = []

Load image paths from `images` diresctory. Later shuffle image paths with a predetermined seed to maintain reproducibility.

In [1]:
import random

RANDOM_SEED = 170081
random.seed(RANDOM_SEED)

In [3]:
import os

In [4]:
def get_image_paths(base_path):
    """Return a list of image files inside the `base_path` """

    image_paths = []
    for (dir_path, _, file_names) in os.walk(base_path):
        for file_name in file_names:
            if os.extsep not in file_name:
                continue
            extension = file_name.split(os.extsep)[-1]
            if extension not in {'jpg'}:
                continue
            if '.ipynb' in dir_path:
                continue
            image_paths.append(os.path.join(dir_path, file_name))

    random.shuffle(image_paths)
    return image_paths

In [5]:
DATASET_DIRECTORY = "../images"

image_paths = get_image_paths(DATASET_DIRECTORY)

In [6]:
image_paths[:5]

['../images/normal-pylorus/c0ffa4e9-ae10-4697-987c-82fcd0e25a9a.jpg',
 '../images/ulcerative-colitis/022d5583-4d26-430b-96b7-cea372edbd3b.jpg',
 '../images/normal-pylorus/40f9b93e-e6bc-4ac8-9da9-ba378ee7f7a1.jpg',
 '../images/normal-cecum/a6043c32-63ff-4819-8e68-8f9c9c29e845.jpg',
 '../images/normal-pylorus/b725d9ef-aec5-4088-9060-fa6788e3a508.jpg']

Preprocess each image and store flattened image(32*32*3=3072) in `data` list and label in `labels` list.

Note: Each image should be in a directory corresponding to the class is belongs to

In [7]:
import numpy as np
import cv2

In [8]:
IMAGE_SIZE = 32

print("[INFO] Started pre-processing")

for i in range(len(image_paths)):
    image_path = image_paths[i]
    
    image = cv2.imread(image_path)
    image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = image.flatten()
    
    label = image_path.split(os.path.sep)[-2]

    image = np.array(image, dtype='float')
    data.append(image / 255.0)
    labels.append(label)
    
    print(f"[INFO] {i}/{len(image_paths)} done", end='\r')

# Scale pixel values to be in between 0-1 instead of 0-255. Also converts them to a numpy array.
data = np.array(data)
labels = np.array(labels)
data = data / 255

print()
print("[INFO] pre-processing completed")

[INFO] Started pre-processing
[INFO] 7999/8000 done
[INFO] pre-processing completed


In [9]:
data

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.68935025e-05, 7.68935025e-05, 7.68935025e-05, ...,
        4.61361015e-05, 4.61361015e-05, 4.61361015e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.92195309e-04, 2.92195309e-04, 2.92195309e-04, ...,
        2.92195309e-04, 2.92195309e-04, 2.92195309e-04],
       [3.07574010e-05, 3.07574010e-05, 3.07574010e-05, ...,
        1.53787005e-05, 1.53787005e-05, 1.53787005e-05],
       [3.07574010e-05, 6.15148020e-05, 6.15148020e-05, ...,
        3.07574010e-05, 3.07574010e-05, 3.07574010e-05]])

In [10]:
labels

array(['normal-pylorus', 'ulcerative-colitis', 'normal-pylorus', ...,
       'normal-pylorus', 'dyed-lifted-polyps', 'ulcerative-colitis'],
      dtype='<U22')

## Split into training and testing sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_x, test_x, train_y, test_y = train_test_split(data, labels, \
                                                    test_size=0.25, \
                                                    random_state=RANDOM_SEED)

Keras will assume that,
* Labels are encoded as integers
* Labels are on-hot encoded

So labels have to be encoded as such. For this scikit learn label binarizer is used. However if this is 2-class only, use Keras' to_categorical function.

In [13]:
from sklearn.preprocessing import LabelBinarizer

In [14]:
label_binarizer = LabelBinarizer()

In [15]:
label_binarizer.fit(labels)

train_y = label_binarizer.transform(train_y)
test_y = label_binarizer.transform(test_y)

In [16]:
train_y[:5]

array([[0, 0, 0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]])

## Defining and Compiling Keras Model Architecture

![Architecture](https://www.pyimagesearch.com/wp-content/uploads/2018/09/keras_tutorial_simplenn_arch.png)

In [17]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [18]:
input_shape = (IMAGE_SIZE*IMAGE_SIZE*3,)
output_size = len(label_binarizer.classes_)

model = Sequential()
model.add(Dense(1024, input_shape=input_shape, activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(output_size, activation='softmax'))

W0816 06:41:47.793372 140648136226624 deprecation_wrapper.py:119] From /home/kdsuneraavinash/Programs/miniconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 06:41:47.972087 140648136226624 deprecation_wrapper.py:119] From /home/kdsuneraavinash/Programs/miniconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 06:41:48.013408 140648136226624 deprecation_wrapper.py:119] From /home/kdsuneraavinash/Programs/miniconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



Compile the model using SGD(Stochastic Gradient Descent) as optimizer and categorical cross-entropy loss (use binary_crossentropy for 2-class classification)

In [19]:
from keras.optimizers import SGD

In [20]:
INITIAL_LEARNING_RATE = 0.01
EPOCHS = 50

optimizer = SGD(lr=INITIAL_LEARNING_RATE)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,\
             metrics=['accuracy'])

W0816 06:41:48.221048 140648136226624 deprecation_wrapper.py:119] From /home/kdsuneraavinash/Programs/miniconda3/envs/tf/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0816 06:41:48.235757 140648136226624 deprecation_wrapper.py:119] From /home/kdsuneraavinash/Programs/miniconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



## Training the Model

In [21]:
print("[INFO] training network")

history = model.fit(x=train_x, \
                    y=train_y, \
                   batch_size=32,\
                   epochs=EPOCHS,\
                   validation_data=(test_x, test_y))

print("[INFO] training completed")

[INFO] training network


W0816 06:41:48.543533 140648136226624 deprecation.py:323] From /home/kdsuneraavinash/Programs/miniconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0816 06:41:48.581713 140648136226624 deprecation_wrapper.py:119] From /home/kdsuneraavinash/Programs/miniconda3/envs/tf/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 6000 samples, validate on 2000 samples
Epoch 1/50
6000/6000 [==============================] - 12s 2ms/step - loss: 2.0939 - acc: 0.1283 - val_loss: 2.0954 - val_acc: 0.1395
Epoch 2/50
6000/6000 [==============================] - 12s 2ms/step - loss: 2.0930 - acc: 0.1278 - val_loss: 2.0810 - val_acc: 0.1310
Epoch 3/50
6000/6000 [==============================] - 7s 1ms/step - loss: 2.0926 - acc: 0.1240 - val_loss: 2.0969 - val_acc: 0.1175
Epoch 4/50
6000/6000 [==============================] - 8s 1ms/step - loss: 2.0911 - acc: 0.1287 - val_loss: 2.1238 - val_acc: 0.1270
Epoch 5/50
6000/6000 [==============================] - 7s 1ms/step - loss: 2.0956 - acc: 0.1187 - val_loss: 2.1030 - val_acc: 0.1220
Epoch 6/50
6000/6000 [==============================] - 8s 1ms/step - loss: 2.0931 - acc: 0.1267 - val_loss: 2.1054 - val_acc: 0.1220
Epoch 7/50
6000/6000 [==============================] - 8s 1ms/step - loss: 2.0933 - acc: 0.1245 - val_loss: 2.1046 - val_acc: 0.1200
Epoch 8/50
6

KeyboardInterrupt: 

## Evaluating the model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predictions = model.predict(test_x, batch_size=32)

correct_predictions = test_y.argmax(axis=1)
real_predictions = predictions.argmax(axis=1)

class_names = label_binarizer.classes_
report = classification_report(correct_predictions, \
                               real_predictions, \
                               target_names=class_names)

print(report)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
PLOT_SAVE_PATH = "simple-nn/evaluation"

epoch_range = np.arange(0, EPOCHS)
plt.style.use('ggplot')
plt.figure()

plt.plot(epoch_range, history.history['loss'], label='Loss(Train)')
plt.plot(epoch_range, history.history['val_loss'], label='Loss(Test)')
plt.plot(epoch_range, history.history['acc'], label='Accuracy(Train)')
plt.plot(epoch_range, history.history['val_acc'], label='Accuracy(Test)')

plt.title("Training Loss and Accuracy (Simple NN)")
plt.xlabel("Epoch Number")
plt.ylabel("Loss/Accuracy")
plt.legend()

plt.savefig(PLOT_SAVE_PATH)

plt.show()

Saving the model in disk.

In [ ]:
MODEL_SAVE_PATH = "simple-nn/model.hdf5"
CLASSES_SAVE_PATH = "simple-nn/classes.txt"

model.save(MODEL_SAVE_PATH)

with open(CLASSES_SAVE_PATH, 'w') as f:
    f.write("\n".join(label_binarizer.classes_))

## Predicting for new data 

In [ ]:
from keras.models import load_model
import cv2

In [ ]:
IMAGE_LOAD_PATH = "../images/dyed-resection-margins/016cc0c1-2a9e-464c-884f-0997561f7dde.jpg"

image = cv2.imread(IMAGE_LOAD_PATH)
image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
image = image.flatten()
image = image.reshape(1, image.shape[0])

model = load_model(MODEL_SAVE_PATH)

classes = []
with open(CLASSES_SAVE_PATH, 'r') as f:
    for line in f.readlines():
        classes.append(line.strip())

prediction = model.predict(image)

pred_i = prediction.argmax(axis=1)[0]
predicted_label = label_binarizer.classes_[pred_i]

In [ ]:
list(zip(label_binarizer.classes_, prediction[0]))

In [ ]:
predicted_label

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
output = cv2.imread(IMAGE_LOAD_PATH)

text = "{}: {:.2f}%".format(predicted_label, prediction[0][pred_i] * 100)

cv2.putText(output, text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2,
	(255, 255, 0), 3)

plt.axis('off')
plt.imshow(output)